In [1]:
!./brax_tutorial.py

Importing NumPy...
Importing JAX...
Importing MuJoCo...
Importing MJX...
^C
Traceback (most recent call last):
  File "/Users/willsturgeon/Desktop/Code/eye-bot-rl/./brax_tutorial.py", line 57, in <module>
    import brax
  File "/usr/local/lib/python3.13/site-packages/brax/__init__.py", line 19, in <module>
    from brax.base import Motion
  File "/usr/local/lib/python3.13/site-packages/brax/base.py", line 23, in <module>
    from flax import struct
  File "/usr/local/lib/python3.13/site-packages/flax/__init__.py", line 26, in <module>
    from flax import linen
  File "/usr/local/lib/python3.13/site-packages/flax/linen/__init__.py", line 39, in <module>
    from .activation import (
    ...<26 lines>...
    )
  File "/usr/local/lib/python3.13/site-packages/flax/linen/activation.py", line 21, in <module>
    from flax.linen.module import compact
  File "/usr/local/lib/python3.13/site-packages/flax/linen/module.py", line 983, in <module>
    class Module(ModuleBase):
    ...<1929 lines>